In [833]:
# import code block
from selenium import webdriver
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
import requests
import csv
import time
import random
import matplotlib.pyplot as plt
from urllib.request import urlopen
import re
from pandas import ExcelWriter

In [834]:
html = urlopen('https://masslandlords.net/policy/eviction-data/?ct=t%28Event+Webinar+2021-01-22+T-21+v1%29')
bs = BeautifulSoup(html, 'html.parser')
# collect all of the dates urls into one list
urls = bs.find_all('a',{'href':re.compile('https://masslandlords.net/policy/eviction-data/filings-week-ending-*')})


In [835]:
# clean urls of unnecessary attribute data - only need href
urls = [url['href'] for url in urls]
print(urls[17])
html = urlopen(urls[17])
bs = BeautifulSoup(html, 'html.parser')

text = bs.find('section',id='main-content').find('p', class_='monospace').get_text()

https://masslandlords.net/policy/eviction-data/filings-week-ending-2021-02-20/


In [836]:
a,b,c,d,e,f =text.split('--')

In [837]:
# remove beginning and ending newlines
reg = re.compile('^\r\n')
c = reg.sub('',c,count=1)
reg2 = re.compile('\r\n\r\n\r\n$')
c=reg2.sub('',c,count=1)
c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11 = c.split("\r\n\r\n\r\n")

In [838]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c1 = reg.sub('',c1,count=1)

# split the table into separate columns
c1 = c1.split('\r\n')
for index in range(1,len(c1)):
    c1[index] = re.split('  +',c1[index])

# load the data into a dataframe
courts = pd.DataFrame(c1[2:],columns=c1[1])
courts

,(n),Count,Percent
0,northeast,88,20.8%
1,western,76,18.0%
2,central,67,15.8%
3,southeast,56,13.2%
4,eastern,51,12.1%
5,metro_south,23,5.4%
6,chelsea district,12,2.8%
7,cambridge district,9,2.1%
8,quincy district,8,1.9%
9,somerville district,7,1.7%


In [839]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c2 = reg.sub('',c2,count=1)

# split the table into separate columns
c2 = c2.split('\r\n')
for index in range(1,len(c2)):
    c2[index] = re.split('  +',c2[index])

# load the data into a dataframe
party_type = pd.DataFrame(c2[2:],columns=c2[1])
party_type

,(n),Plaintiffs,Defendants
0,Corporate Entity,315,2
1,Natural Person,108,421
2,Total,423,423


In [840]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c3 = reg.sub('',c3,count=1)

# split the table into separate columns
c3 = c3.split('\r\n')
for index in range(0,len(c3)):
    c3[index] = re.split('  +',c3[index])

# load the data into a dataframe
party_type_pct = pd.DataFrame(c3[1:],columns=c3[0])
party_type_pct

,(%),Plaintiffs,Defendants
0,Corporate Entity,74.5%,0.5%
1,Natural Person,25.5%,99.5%
2,Total,100.0%,100.0%


In [841]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c4 = reg.sub('',c4,count=1)

# split the table into separate columns
c4 = c4.split('\r\n')
for index in range(0,len(c4)):
    c4[index] = re.split('  +',c4[index])

# load the data into a dataframe
plaintiff_repr = pd.DataFrame(c4[2:],columns=c4[1])
plaintiff_repr

,(n),Has Attorney,Pro Se,Total
0,Required,311,4,315
1,Optional,48,60,108
2,Total,359,64,423


In [842]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c5 = reg.sub('',c5,count=1)

# split the table into separate columns
c5 = c5.split('\r\n')
name = c5[0]
for index in range(0,len(c5)):
    c5[index] = re.split('  +',c5[index])

# load the data into a dataframe
plaintiff_repr_pct = pd.DataFrame(c5[1:],columns=c5[0])
plaintiff_repr_pct

,(%),Has Attorney,Pro Se,Total
0,Required,73.5%,0.9%,74.5%
1,Optional,11.3%,14.2%,25.5%
2,Total,84.9%,15.1%,100.0%


In [843]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c6 = reg.sub('',c6,count=1)

# split the table into separate columns
c6 = c6.split('\r\n')
for index in range(0,len(c6)):
    c6[index] = re.split('  +',c6[index])

# load the data into a dataframe, cutting off note: "Defendants will not have an attorney known to the plaintiff at time of filing unless this is an ongoing matter. 100% pro se is to be expected in all filing reports except to the extent time passes between filing and staff review. See outcomes report for final assessment."
defendant_repr = pd.DataFrame(c6[4:],columns=c6[3])
defendant_repr

,(n),Has Attorney,Pro Se,Total
0,Required,0,2,2
1,Optional,14,407,421
2,Total,14,409,423


In [844]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c7 = reg.sub('',c7,count=1)

# split the table into separate columns
c7 = c7.split('\r\n')
for index in range(0,len(c7)):
    c7[index] = re.split('  +',c7[index])

# load the data into a dataframe
defendant_repr_pct = pd.DataFrame(c7[1:],columns=c7[0])
defendant_repr_pct

,(%),Has Attorney,Pro Se,Total
0,Required,0.0%,0.5%,0.5%
1,Optional,3.3%,96.2%,99.5%
2,Total,3.3%,96.7%,100.0%


In [845]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c8 = reg.sub('',c8,count=1)

# split the table into separate columns
c8 = c8.split('\r\n')
for index in range(len(c8)):
    c8[index] = re.split('  +',c8[index])

# load the data into a dataframe
num_adults = pd.DataFrame(c8[2:-1],columns=c8[0])
num_adults

,Number of Adults in Households,Count,Percent
0,1,300,70.9%
1,2,72,17.0%
2,3,9,2.1%
3,4,1,0.2%
4,5,3,0.7%
5,Total,423,100.0%


In [846]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c9 = reg.sub('',c9,count=1)

# split the table into separate columns
c9 = c9.split('\r\n')
for index in range(len(c9)):
    c9[index] = re.split('  +',c9[index])

# load the data into a dataframe
init_action = pd.DataFrame(c9[1:],columns=c9[0])
init_action

,Initiating Action,Count,Percent
0,Non-payment,272,64.3%
1,Cause,69,16.3%
2,No Fault,50,11.8%
3,Unknown/Other,30,7.1%
4,Foreclosure,2,0.5%


In [847]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c10 = reg.sub('',c10,count=1)

# split the table into separate columns
c10 = c10.split('\r\n')
for index in range(len(c10)):
    c10[index] = re.split('  +',c10[index])

# load the data into a dataframe
rate_per_100k = pd.DataFrame(c10[2:-1],columns=c10[1])
rate_per_100k

,Municipality,Residents,Count
0,Chestnut Hill,188,4
1,East Falmouth,69,4
2,Pocasset,34,1
3,Dennis Port,31,1
4,Millville,31,1
...,...,...,...
60,Plymouth,1,1
61,South Weymouth,1,1
62,Waltham,1,1
63,(not given),0,211


In [848]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
c11 = reg.sub('',c11,count=1)

# split the table into separate columns
c11 = c11.split('\r\n')
for index in range(len(c11)):
    c11[index] = re.split('  +',c11[index])

# load the data into a dataframe
rate_per_100k_renters = pd.DataFrame(c11[2:-2],columns=c11[1])
rate_per_100k_renters

,County,Renter Households,Count
0,Barnstable,56,11
1,Plymouth,55,24
2,Bristol,54,45
3,Berkshire,37,6
4,Norfolk,28,24
5,Suffolk,27,54
6,Worcester,16,18
7,Middlesex,11,27
8,Essex,2,3
9,Dukes,0,0


In [849]:
# remove beginning and ending newlines
reg = re.compile('^\r\n')
d = reg.sub('',d,count=1)
reg2 = re.compile('\r\n\r\n\r\n$')
reg3=re.compile('\r\n\r\n$')
d=reg2.sub('',d,count=1)
d1,d2= d.split("\r\n\r\n\r\n")
d1,d2,d3,d4,d5,d6,d7=d2.split("\r\n\r\n")

In [850]:
# remove newline at the beginning of the table "block"
reg = re.compile('^\r\n')
d1 = reg.sub('',d1,count=1)

# split the table into separate columns
d1 = d1.split('\r\n')
for index in range(0,len(d1)):
    d1[index] = re.split('  +',d1[index])

# load the data into a dataframe
County_household_percent_renters= pd.DataFrame(d1[1:],columns=['County', 'Households', '% Renters'])
County_household_percent_renters

,County,Households,% Renters
0,Barnstable,"96,509",20.0%
1,Berkshire,"53,792",30.0%
2,Bristol,"220,528",37.3%
3,Dukes,"18,146",22.6%
4,Essex,"297,898",36.9%
5,Franklin,"30,927",34.1%
6,Hampden,"179,970",39.5%
7,Hampshire,"60,002",30.7%
8,Middlesex,"612,366",38.1%
9,Nantucket,"11,399",30.8%


In [851]:
#name file 
name=re.sub('\/', '', urls[17])
name=name[50:]
print(name)

week-ending-2021-02-20


In [852]:
#pip install xlsxwriter
#pip install xlwx
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r"C:\Users\wjg39\Documents\Desktop\AP Evictions\\"+name+".xlsx")

In [853]:
#write each DataFrame to a specific sheet
courts.to_excel(writer, sheet_name='Courts',index=False)
party_type.to_excel(writer, sheet_name='Party Type',startrow=0 , startcol=0,index=False)
party_type_pct.to_excel(writer, sheet_name='Party Type',startrow=6 , startcol=0,index=False)
plaintiff_repr.to_excel(writer, sheet_name='Plaintiff Representation',startrow=0 , startcol=0,index=False)
plaintiff_repr_pct.to_excel(writer, sheet_name='Plaintiff Representation',startrow=6 , startcol=0,index=False)
defendant_repr.to_excel(writer, sheet_name='Defendant Representation',startrow=0 , startcol=0,index=False)
defendant_repr_pct.to_excel(writer, sheet_name='Defendant Representation',startrow=6 , startcol=0,index=False)
num_adults.to_excel(writer, sheet_name='Number of Adults in Households',index=False)
init_action.to_excel(writer, sheet_name='Initiating Action',index=False)
rate_per_100k.to_excel(writer, sheet_name='Municipality rate per 100k',index=False)
rate_per_100k_renters.to_excel(writer, sheet_name='County rate per 100k',index=False)
County_household_percent_renters.to_excel(writer, sheet_name='County Households % Renters',index=False)

#close the Pandas Excel writer and output the Excel file
writer.save()